#  Projet Text to Handwriting – Préparation du Dataset IAM
Ce notebook prépare les données du dataset **IAM Handwriting Database** pour un projet de génération d'écriture manuscrite (Text to Handwriting) avec PyTorch.

##  1. Importation des bibliothèques et vérification des dossiers

In [37]:
# Importer les bibliothèques nécessaires
import os
import pandas as pd
import cv2
import numpy as np
from sklearn.model_selection import train_test_split

# Définir le chemin du dossier IAM
data_dir = "IAM_dataset"
lines_file = os.path.join(data_dir, "ascii", "lines.txt")
lines_img_dir = os.path.join(data_dir, "lines")  # Dossier des images

# Vérifier si les dossiers existent
if not os.path.exists(data_dir):
    print("❌ Erreur : Le dossier IAM_dataset n'existe pas.")
elif not os.path.exists(lines_img_dir):
    print("❌ Erreur : Le dossier lines n'existe pas dans IAM_dataset.")
else:
    print(f"✅ Dossier trouvé : {data_dir}")
    print(f"📁 Dossier des images : {lines_img_dir}")

✅ Dossier trouvé : IAM_dataset
📁 Dossier des images : IAM_dataset\lines


##  2. Lecture et parsing du fichier `lines.txt`
On extrait le chemin d'image et le texte correspondant pour chaque ligne du dataset.

In [38]:
# Lecture et parsing du fichier lines.txt
lines_data = []

with open(lines_file, 'r', encoding='utf-8') as f:
    for i, line in enumerate(f):
        if line.startswith('#') or not line.strip():
            continue

        parts = line.strip().split()
        if len(parts) >= 9:
            img_id = parts[0]
            status = parts[1]

            if status == 'ok':
                text = ' '.join(parts[8:]).replace('|', ' ')
                img_base = img_id.split('-')[0]
                sub_dir = '-'.join(img_id.split('-')[:2])
                img_name = f"{img_id}.png"
                img_path = os.path.join(lines_img_dir, img_base, sub_dir, img_name)

                if os.path.exists(img_path):
                    lines_data.append((img_path, text))
                else:
                    print(f"⚠️ Image non trouvée : {img_path}")

df_lines = pd.DataFrame(lines_data, columns=['image_path', 'text'])
print(f"\nNombre total de lignes valides : {len(df_lines)}")
print(df_lines.head())


Nombre total de lignes valides : 11344
                                       image_path  \
0  IAM_dataset\lines\a01\a01-000u\a01-000u-00.png   
1  IAM_dataset\lines\a01\a01-000u\a01-000u-01.png   
2  IAM_dataset\lines\a01\a01-000u\a01-000u-02.png   
3  IAM_dataset\lines\a01\a01-000u\a01-000u-04.png   
4  IAM_dataset\lines\a01\a01-000u\a01-000u-06.png   

                                        text  
0          A MOVE to stop Mr. Gaitskell from  
1      nominating any more Labour life Peers  
2       is to be made at a meeting of Labour  
3       put down a resolution on the subject  
4  Griffiths , M P for Manchester Exchange .  


##  3. Préparation du Dataset PyTorch

In [39]:
from PIL import Image
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader

# Définir les transformations (ex : 128x32, niveaux de gris)
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.Resize((32, 128)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Définir un Dataset personnalisé
class HandwrittenDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.dataframe = dataframe
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_path = self.dataframe.iloc[idx]['image_path']
        text = self.dataframe.iloc[idx]['text']
        image = Image.open(img_path).convert('L')

        if self.transform:
            image = self.transform(image)

        return image, text

# Créer le DataLoader
dataset = HandwrittenDataset(df_lines, transform=transform)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# Vérification d'un batch
for batch_imgs, batch_texts in dataloader:
    print(f"Batch images shape : {batch_imgs.shape}")
    print(f"Exemple de texte : {batch_texts[0]}")
    break

Batch images shape : torch.Size([32, 1, 32, 128])
Exemple de texte : palms of his hands .


##  4. Résumé de la préparation
- Vérification du dossier IAM
- Parsing de `lines.txt`
- Association des chemins d'images et des textes
- Préparation d’un DataLoader utilisable pour l’entraînement d’un modèle IA